In [ ]:
import os
import numpy as np
import pickle as pkl
from openai import OpenAI

## Load Dataset

In [ ]:
city = 'ny'

city_full_name = {
    'ny': 'New York City',
    'hs': 'Houston',
    'sf': 'San Francisco'
}

In [ ]:
with open('indices.pkl', 'rb') as f:
    indices = pkl.load(f)
    
with open('dates.pkl', 'rb') as f:
    dates = pkl.load(f)
    
with open(f'time_series_{city}.pkl', 'rb') as f:
    data = pkl.load(f)

In [ ]:
data_size = data.shape[0]
window_size = 24
print(data_size, window_size, len(indices))

In [ ]:
data_size = len(indices)

num_train = int(data_size * 0.6)
num_test = int(data_size * 0.2)
num_vali = data_size - num_train - num_test

seq_len_day = 1

idx_train = np.arange(num_train - seq_len_day)
idx_valid = np.arange(num_train - seq_len_day, num_train + num_vali - seq_len_day)
idx_test = np.arange(num_train + num_vali - seq_len_day, num_train + num_vali + num_test - seq_len_day)

## Prompt GPT 4

In [ ]:
API_KEY = ''

In [ ]:
system_prompt = f"Your job is to act as a professional weather forecaster. You will be given a time-series data of the weather from the past 24 hours. Based on this information, your task is to predict whether it will rain in the next 24 hours."
print(system_prompt)

In [ ]:
client = OpenAI(api_key=API_KEY)

In [ ]:
for _i in idx_test:
    i = indices[_i]
    
    data_window = data[i:i+window_size]
    
    humidity = '|'.join([f"{x:.2f}" for x in data_window[:,0]])
    pressure = '|'.join([f"{x:.2f}" for x in data_window[:,1]])
    temperature = '|'.join([f"{x:.2f}" for x in data_window[:,2]])
    wind_speed = '|'.join([f"{x:.2f}" for x in data_window[:,3]])
    wind_direction = '|'.join([f"{x:.2f}" for x in data_window[:,4]])
    
    user_prompt = f"Your task is to predict whether it will rain or not in {city_full_name[city]} in the next {window_size} hours. "
    user_prompt += f"Review the time-series data provided for the last {window_size} hours. "
    user_prompt += f"Each time-series consists of hourly values separated by a \'|\' token for the following indicators:\n\n"
    user_prompt += f"- Temperature (Kelvin): {temperature}\n- Humidity (%): {humidity}\n- Air Pressure (hPa): {pressure}\n- Wind Speed (m/s): {wind_speed}\n- Wind Direction (degrees): {wind_direction}\n\n"
    user_prompt += f"Based on this information, respond with either \'rain\' or \'not rain\'. Do not provide any other details. "
    
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
        {
          "role": "system",
          "content": system_prompt
        },
        {
          "role": "user",
          "content": user_prompt
        }
        ],
        temperature=0.7,
        max_tokens=2048,
        top_p=1
    )

    text = response.choices[0].message.content

    with open(f'gpt_predict/{city}_{i}.txt', 'w') as f:
        f.write(text)